# Hilti explorer

> Ayush Baid, Frank Dellaert

A notebook to investigate the hilti dataset

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import dask
import plotly.express as px
import gtsam
from gtsam import Cal3Bundler, EssentialMatrix, Point3, Pose3, Rot3, Unit3
from gtbook.drone import axes
import hydra
from dask.distributed import Client, LocalCluster
from hydra.utils import instantiate
from omegaconf import OmegaConf


import gtsfm.utils.io as io_utils
from gtsfm.loader.hilti_loader import HiltiLoader
from gtsfm.common.gtsfm_data import GtsfmData
from gtsfm.common.image import Image
from gtsfm.common.keypoints import Keypoints
from gtsfm.frontend.detector_descriptor.sift import SIFTDetectorDescriptor
from gtsfm.frontend.matcher.twoway_matcher import TwoWayMatcher
from gtsfm.utils import viz
from gtsfm.frontend.verifier.ransac import Ransac
from gtsfm.scene_optimizer import SceneOptimizer
from gtsfm.retriever.sequential_hilti_retriever import SequentialHiltiRetriever
from gtsfm.two_view_estimator import TwoViewEstimator
import gtsfm.utils.geometry_comparisons as comp_utils

In [ ]:
cwd = Path.cwd()
folder_path = Path(cwd.parent / "tests" / "data" / "hilti_exp4_medium")
print(folder_path)

In [ ]:
%ls /Users/dellaert/git/gtsfm/tests/data/hilti_exp4_medium

In [ ]:
# Load Images
indices = range(300, 300+20)
loader = HiltiLoader(base_folder=str(folder_path))
images = [loader.get_image(i) for i in indices]

In [ ]:
translations = np.array([loader.get_absolute_pose_prior(i).value.translation() for i in range(len(loader))])
print(translations.shape)

In [ ]:
wTimu0 = Pose3(Rot3(Point3(1, 0, 0), Point3(0, -1, 0), Point3(0, 0, -1)), Point3(0, 0, 0))

In [ ]:
rig_indices = range(0, 250)
wTimu = {i: (wTimu0 * loader._w_T_imu[i]) for i in rig_indices}
translations = np.array([t.translation() for t in wTimu.values()])

# print(poses)
fig = px.scatter_3d(x=translations[:, 0], y=translations[:, 1], z=translations[:, 2])
for t in wTimu.values():
    fig.add_traces(axes(t))

fig.add_traces(axes(Pose3()))

# camera = dict(
#     up=dict(x=0, y=0, z=-1),
#     # center=dict(x=0, y=0, z=0),
#     # eye=dict(x=1.25, y=1.25, z=1.25)
# )

fig.update_traces(marker=dict(size=3))
fig.show()

In [ ]:
with hydra.initialize_config_module(config_module="gtsfm.configs"):
    # config is relative to the gtsfm module
    cfg = hydra.compose(
        config_name="deep_front_end",
    )
    # print(cfg)
    scene_optimizer: SceneOptimizer = instantiate(cfg.SceneOptimizer)

retriever = SequentialHiltiRetriever(max_frame_lookahead=1)


In [ ]:
rig_indices_filter = range(60, 65)
image_indices_filter = range(300, 300 + 20)

In [ ]:
# create dask client
cluster = LocalCluster(
    n_workers=1, threads_per_worker=1
)

In [ ]:
pairs_graph = retriever.create_computation_graph(loader)
with Client(cluster):
    image_pair_indices = pairs_graph.compute()

In [ ]:
subset_pair_indices = [(i1, i2) for (i1, i2) in image_pair_indices if i1 in image_indices_filter and i2 in image_indices_filter]
print(subset_pair_indices)

In [ ]:
i2Ri1_graph, i2Ui1_graph = scene_optimizer.create_computation_graph_for_frontend(
    image_pair_indices=subset_pair_indices, 
    image_graph=loader.create_computation_graph_for_images(),
    all_intrinsics=loader.create_computation_graph_for_intrinsics(),
    image_shapes = loader.create_computation_graph_for_image_shapes(),
    relative_pose_priors = loader.get_relative_pose_priors(subset_pair_indices),
    gt_poses_graph = loader.create_computation_graph_for_poses())

In [ ]:
dask_input = dask.delayed([i2Ri1_graph, i2Ui1_graph])
with Client(cluster):
    i2Ri1_dict, i2Ui1_dict = dask_input.compute()

In [ ]:
def debug_relative_rotation(i1, i2):
    i2Ri1_sample = i2Ri1_dict[(i1, i2)]
    print("recovered i1Ri2 matrix")
    print(np.round(i2Ri1_sample.inverse().matrix(), 2))
    print("recovered i1Ri2 xyz")
    print(np.round(np.degrees(i2Ri1_sample.inverse().xyz()), 2))

    print("prior i1Ri2")
    print(np.round(loader.get_relative_pose_prior(i1, i2).value.inverse().rotation().matrix(), 2))
    print(np.round(np.degrees(loader.get_relative_pose_prior(i1, i2).value.inverse().rotation().xyz()), 2))

In [ ]:
# confirm the rotation from front-right (300) to right (303)
debug_relative_rotation(300, 303)

In [ ]:
# confirm the rotation from front-left (301) to front-left (306)
debug_relative_rotation(301, 306)

In [ ]:
# confirm the rotation from front-right (300) to front-right (305)
debug_relative_rotation(300, 305)

In [ ]:
# evaluate 2-view relative rotations
for (i1, i2), i2Ri1 in i2Ri1_dict.items():
    i2Ri1_prior = loader.get_relative_pose_prior(i1, i2).value.rotation()

    R_angular_error = comp_utils.compute_relative_rotation_angle(i2Ri1, i2Ri1_prior)
    if R_angular_error is not None and R_angular_error < 1:
        print(i1, i2, R_angular_error)

In [ ]:
i2Ri1_input_hacked = dict(i2Ri1_dict)

hacked_pairs = [(300, 302), (305, 307), (310, 312), (315, 317)]
for (i1, i2) in hacked_pairs:
    i2Ri1_from_prior = loader.get_relative_pose_prior(i1, i2).value.rotation()
    i2Ri1_input_hacked[(i1, i2)] = i2Ri1_from_prior

In [ ]:
import importlib
import gtsfm.averaging.rotation.shonan as shonan_
importlib.reload(shonan_)
shonan = shonan_.ShonanRotationAveraging()
wRi_shonan = shonan.run(320, i2Ri1_dict=i2Ri1_input_hacked, i2Ti1_priors=[])[300:320]

In [ ]:
for i, wRi in enumerate(wRi_shonan):
    print(300 + i, np.round(wRi.matrix(), 2))

In [ ]:
# evaluate 2-view relative rotations
for (i1, i2), i2Ri1 in i2Ri1_dict.items():
    i2Ri1_prior = loader.get_relative_pose_prior(i1, i2).value.rotation()

    R_angular_error_2view = comp_utils.compute_relative_rotation_angle(i2Ri1, i2Ri1_prior)
    if R_angular_error_2view is None: continue

    i2Ri1_shonan = wRi_shonan[i2-300].between(wRi_shonan[i1-300])

    R_angular_error_shonan = np.round(comp_utils.compute_relative_rotation_angle(i2Ri1_shonan, i2Ri1_prior), 2)

    print(i1, i2, np.round(R_angular_error_2view, 2), R_angular_error_shonan, np.round((-R_angular_error_2view+R_angular_error_shonan), 2))


In [ ]:

wRi_gt = [loader.get_camera_pose(i).rotation() for i in range(300, 320)]

print(len(wRi_shonan))
print(len(wRi_gt))

wRi_shonan_aligned = comp_utils.align_rotations(wRi_gt, wRi_shonan)
error = [
    comp_utils.compute_relative_rotation_angle(aRi, aRi_)
    for (aRi, aRi_) in zip(wRi_shonan_aligned, wRi_gt)
]
print(error)

In [ ]:
# Detect features
detector_descriptor = SIFTDetectorDescriptor()
features = [detector_descriptor.detect_and_describe(image) for image in images]
for i,(f,d) in enumerate(features):
    print(f"image {i+1}: {d.shape[0]} features")

In [ ]:
# Do matching
matcher = TwoWayMatcher(ratio_test_threshold=0.8)
keypoints_i1, descriptors_i1 = features[0]
keypoints_i2, descriptors_i2 = features[1]
image_shape_i1 = images[0].value_array.shape
image_shape_i2 = images[1].value_array.shape
match_indices = matcher.match(
    keypoints_i1, keypoints_i2, descriptors_i1, descriptors_i2, image_shape_i1, image_shape_i2
)
print(f"{match_indices.shape[0]} matched.")

In [ ]:
# Get intrinsics
camera_intrinsics_i1, camera_intrinsics_i2, *others = [loader.get_camera_intrinsics_full_res(i) for i in indices]
print(camera_intrinsics_i1)

In [ ]:
# Do verification
verifier = Ransac(use_intrinsics_in_verification=False,
                  estimation_threshold_px=2)
i2Ri1, i2Ui1, v_corr_idxs, inlier_ratio_est_model = verifier.verify(
    keypoints_i1, keypoints_i2, match_indices, camera_intrinsics_i1, camera_intrinsics_i2)
print(f"ypr={np.degrees(i2Ri1.xyz())}\nU={i2Ui1.point3().T}\nverified:{v_corr_idxs.shape}\n{inlier_ratio_est_model=}")

In [ ]:
correspondence_image = viz.plot_twoview_correspondences(*images[:2], keypoints_i1, keypoints_i2, v_corr_idxs)
fig = plt.figure(figsize=(12, 18), dpi=80)
fig.gca().imshow(correspondence_image.value_array)
plt.show()

In [ ]:
# Create estimator
two_view_estimator = TwoViewEstimator(
    matcher=None, verifier=None, inlier_support_processor=None,
    bundle_adjust_2view=True, eval_threshold_px=4
)

In [ ]:
i2Ri1_optimized, i2Ui1_optimized, corr_idxs = two_view_estimator.bundle_adjust(
    keypoints_i1, keypoints_i2, v_corr_idxs, camera_intrinsics_i1, camera_intrinsics_i2, i2Ri1, i2Ui1)
print(f"ypr={np.degrees(i2Ri1_optimized.xyz())}\nU={i2Ui1_optimized.point3().T}\nverified:{corr_idxs.shape}")

In [ ]:
correspondence_image = viz.plot_twoview_correspondences(*images, keypoints_i1, keypoints_i2, corr_idxs)
fig = plt.figure(figsize=(12, 18), dpi=80)
fig.gca().imshow(correspondence_image.value_array)
plt.show()